# Run Synsat for ERA5 Data

Here, we will 
- load ERA5 test data 
- run Synsat on ERA5 and
- finally store the Synsats into a netcdf file

## Setup Env and Load Libraries 

In [1]:
import os, sys
os.environ['RTTOV_PYTHON_WRAPPER'] = '/work/bb1262/tools/rttov/rttov-v13.2/wrapper'

In [2]:
import synsatipy

from synsatipy.synsat import SynSat

import synsatipy.synsat_example_data as synsat_example_data

no git hash can be obtained


## Steps towards Synsat 

### Step I: Initialize Synsat class

In [3]:
s = SynSat( )

... [synsat] set cloud / aerosol file to  /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/cldaer_visir/sccldcoef_msg_3_seviri.dat
... [synsat] load coefficient file /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/rttov13pred54L/rtcoef_msg_3_seviri_o3.dat


 2024/03/26  17:11:19  Load coefficients:
Load successful >>>>> inst_id : 1, nchannels : 6.
 2024/03/26  17:11:19  /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/rttov13pred54L/rtcoef_msg_3_seviri_o3.dat
 2024/03/26  17:11:19  /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/cldaer_visir/sccldcoef_msg_3_seviri.dat


### Step II: Load Example Data 

In [4]:
eraname = synsat_example_data.get_example_data( 'era01' )
s.load( eraname, isel = {'time':[23,]} )

... [synsat] read data from file  /work/bb1262/data/era5/medi/2020/era5-3d-medi-2020-09-15.nc


/home/b/b380352/proj/2022-01_RTTOV-Synsat/synsatipy-devel/synsatipy/data_handler.py:172: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nlevels = profs.dims["lev"]
/home/b/b380352/proj/2022-01_RTTOV-Synsat/synsatipy-devel/synsatipy/data_handler.py:173: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nprofiles = profs.dims["profile"]
/home/b/b380352/proj/2022-01_RTTOV-Synsat/synsatipy-devel/synsatipy/data_handler.py:73: RuntimeWarning: invalid value encountered in arccos
  azi = np.arccos(np.sin(lat) / np.sin(delta))


The ERA5 data have been loaded into a data handler. You can inspect the data as follows:

In [5]:
era5 = s.synsat.data_handler.input_data

In [6]:
print( era5 )

<xarray.Dataset> Size: 94MB
Dimensions:  (time: 1, lon: 213, lat: 61, lev: 137, nhyi: 138, nhym: 137,
              lev_2: 1)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-09-15T23:00:00
  * lon      (lon) float64 2kB -8.0 -7.75 -7.5 -7.25 ... 44.25 44.5 44.75 45.0
  * lat      (lat) float64 488B 30.0 30.25 30.5 30.75 ... 44.25 44.5 44.75 45.0
  * lev      (lev) float64 1kB 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
  * lev_2    (lev_2) float64 8B 1.0
Dimensions without coordinates: nhyi, nhym
Data variables: (12/37)
    SSTK     (time, lat, lon) float32 52kB dask.array<chunksize=(1, 61, 213), meta=np.ndarray>
    SLT      (time, lat, lon) float32 52kB dask.array<chunksize=(1, 61, 213), meta=np.ndarray>
    Z        (time, lat, lon) float32 52kB dask.array<chunksize=(1, 61, 213), meta=np.ndarray>
    SP       (time, lat, lon) float32 52kB dask.array<chunksize=(1, 61, 213), meta=np.ndarray>
    TCW      (time, lat, lon) float32 52kB dask.array<chunksize=(1, 61, 213), meta

In [7]:
era5.time

<xarray.DataArray 'time' (time: 1)> Size: 8B
array(['2020-09-15T23:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-09-15T23:00:00
Attributes:
    standard_name:  time
    axis:           T

### Step III: Setup Run Options and Start Execution 

In [8]:
s._options.Nthreads=4
s._options.NprofsPerCall = 1000

In [9]:
s.run()

IR emissivity atlas loaded successfully
Atlas deallocated.
 2024/03/26  17:11:23  Running RTTOV using nthreads =    4 and nprofs_per_call =     1000


### Step IV: Extract the Output

In [10]:
era_synsat = s.extract_output()

... [synsat]: WARNING: fail to write global attributes


In [11]:
print( era_synsat )

<xarray.Dataset> Size: 626kB
Dimensions:  (time: 1, lon: 213, lat: 61)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-09-15T23:00:00
  * lon      (lon) float64 2kB -8.0 -7.75 -7.5 -7.25 ... 44.25 44.5 44.75 45.0
  * lat      (lat) float64 488B 30.0 30.25 30.5 30.75 ... 44.25 44.5 44.75 45.0
Data variables:
    bt062    (time, lon, lat) float64 104kB 229.8 229.4 230.6 ... 233.9 233.6
    bt073    (time, lon, lat) float64 104kB 242.0 239.3 246.3 ... 252.5 251.9
    bt087    (time, lon, lat) float64 104kB 260.0 251.1 269.2 ... 282.3 282.6
    bt108    (time, lon, lat) float64 104kB 257.7 249.3 268.2 ... 285.3 285.5
    bt120    (time, lon, lat) float64 104kB 255.1 247.6 265.6 ... 284.4 284.6
    bt134    (time, lon, lat) float64 104kB 244.6 240.1 251.9 ... 259.3 259.1
Attributes:
    CDI:                       Climate Data Interface version 2.0.5 (https://...
    Conventions:               CF-1.6
    institution:               European Centre for Medium-Range Weather Forec...
   

### Step V: Store Data 

In [12]:
outdir = './Data'

if not os.path.isdir( outdir ):
    os.makedirs( outdir )
outfile = f'{outdir}/synsat_era5_example_data.nc'

print(f'... store synsat at {outfile}')
era_synsat.to_netcdf(outfile)


... store synsat at ./Data/synsat_era5_example_data.nc
